<a href="https://colab.research.google.com/github/MarinaZRocha/SI_2023/blob/main/Autogen_Vicuna_13B_Exemplo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de AutoGen com modelo Vicuna rodando localmente

Neste exemplo abaixo mostramos como rodar um sistema multiagentes usando autogen com modelo Vicuna de 13 bilhões de parâmetros rodando localmente no computador do Google Colab.

vicuna13b.svg


Como pode ser observado no gráfico acima, o modelo Vicuna-13B apresenta performance comparável à do modelo Bard, um pouco abaixo da performance do ChatGPT-4 tomado como referência.

# Passo 1: Instalação do modelo LLaMA em C/C++

A linha de código abaixo instala o servidor LLaMA baseado em código em C/C++ que roda de forma mais eficiente, consumindo menos memória que a versão nativa em Python.

In [1]:
%%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python[server]

# Passo 2: Instalação do AutoGen

AutoGen é o sistema que permite a construção de aplicações que envolvem conversação entre agentes baseados em modelos linguísticos de larga escala (LLMs).

In [2]:
%%capture
!pip3 install pyautogen

# Passo 3: Instalação do Vicuna 13B

Nas linhas abaixo configuramos e instalamos o modelo Vicuna de 13 bilhões de parâmetros. Esse modelo vai rodar localmente no computador do Google Colab usando LLaMA C/C++. Esse será o modelo usado pelo AutoGen.

In [3]:
model_name = "vicuna-13b-v1.5.Q4_0.gguf"
repo = "TheBloke/vicuna-13B-v1.5-GGUF"
chat_format = "vicuna"

In [4]:
%%capture
!wget https://huggingface.co/{repo}/resolve/main/{model_name}?download=true -O {model_name}

# Passo 4: Rodando o servidor LLaMA C/C++

A linha abaixo inicia o serviço do servidor LLaMA C/C++ que seguirá rodando no plano de fundo.

In [1]:
!nohup python -m llama_cpp.server --model {model_name} --n_gpu_layers 100 --n_ctx 4096 --host 127.0.0.1 --chat_format {chat_format} &

nohup: appending output to 'nohup.out'


O comando abaixo verifica se o servidor está ainda rodando. Se não estiver, verifique passo acima.

In [6]:
!ps -eFww | grep llama_cpp | grep grep -v

root        3446       1 38 3435243 873332 1 14:55 ?       00:00:02 python3 -m llama_cpp.server --model vicuna-13b-v1.5.Q4_0.gguf --n_gpu_layers 100 --n_ctx 4096 --host 127.0.0.1 --chat_format vicuna


In [7]:
!kill -KILL 11387

/bin/bash: line 1: kill: (11387) - No such process


# Passo 5: Configuração do AutoGen

Nas linhas abaixo importamos os módulos do Python para AutoGen e configuramos o API para utilização do Vicuna através do servidor LLaMA C/C++ rodando na própria máquina.

In [8]:
import autogen
from autogen import oai

In [9]:
config_list=[
  {
    "model": model_name,
    "base_url": "http://127.0.0.1:8000/v1",
    #"api_base": "http://127.0.0.1:8000/v1",
    #"api_type": "open_ai",
    "api_key": "NULL", # just a placeholder
  }
]

# Passo 7: Exemplo

Começamos configurando detalhes de tempo máximo de resposta, semente para a geração aleatória, e a variável config_list do passo 5 acima.

In [10]:
llm_config={
  #"request_timeout": 600,
  "seed": 5432,
  "config_list": config_list,
  "temperature": 0
}

No exemplo abaixo criamos um assistente que assume o papel de um CTO de uma empresa de tecnologia. Esse cargo representa o diretor-chefe da área de tecnologia de uma empresa.

In [14]:
assistant = autogen.AssistantAgent(
  name="CTO",
  llm_config=llm_config,
  system_message="Chief technical officer of a tech company"
)

As regras abaixo definem como o sistema deve sinalizar se a tarefa solicitada foi resolvida.

In [15]:
assistant.update_system_message(assistant.system_message+"""Reply TERMINATE if the task has been solved at full satisfaction.
  Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""")

Agora define-se o agente que será o proxy do usuário humano. Esse agente vai responder em nome do humano por até 5 respostas consecutivas.

In [17]:
user_proxy = autogen.UserProxyAgent(
  name="user_proxy",
  human_input_mode="TERMINATE",
  max_consecutive_auto_reply=5,
  is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
  code_execution_config={"work_dir": "web"},
  llm_config=False,
)

Finalmente pedimos uma tarefa abaixo. Aqui pedimos que escreva código em Python para escrever os números de 1 a 100.

In [ ]:
# @title Default title text
task = """
What color is the sky?
"""
user_proxy.initiate_chat(
  assistant,
  message=task
)

user_proxy (to CTO):


What color is the sea?


--------------------------------------------------------------------------------
CTO (to user_proxy):

 The color of the sea can vary depending on various factors such as the depth, the presence of plankton or other substances, and the time of day. Generally, the sea appears blue because of the way it reflects the sun's light. However, it can also appear green, gray, or even black in certain conditions.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
user_proxy (to CTO):



--------------------------------------------------------------------------------
CTO (to user_proxy):

  The color of the sea can vary depending on various factors such as the depth, the presence of plankton or other substances, and the time of day. Generally, the sea appears blue because of the way it reflects the sun's light. However, it can also appear green, gray, or even black in certain conditions.



PS.: No exemplo acima o assistente realizou a tarefa mas "esqueceu" de colocar a palavra-chave TERMINATE para indicar que a tarefa terminou.